In [1]:
import pandas as pd
import json
import xlsxwriter

In [2]:
raw_df = pd.read_excel("./Project_data_final.xlsx")

In [3]:
raw_df

,sEth,dEth,typEth,Source,Destination,proto,sPort,dPort,priority,action
0,9e:8d:de:80:29:28,*,*,*,dcape-na.amazon.com,6,*,443,*,forward
1,9e:8d:de:80:29:28,*,*,*,softwareupdates.amazon.com,6,*,443,*,forward
2,9e:8d:de:80:29:28,*,*,*,3.north-america.pool.ntp.org,17,*,123,*,forward
3,9e:8d:de:80:29:28,*,*,*,224.0.0.22/32,2,*,*,*,forward
4,9e:8d:de:80:29:28,*,*,*,239.255.255.250/32,17,*,1900,*,forward
...,...,...,...,...,...,...,...,...,...,...
690,9e:8d:de:80:29:28,*,*,*,scalews.withings.net,6,*,443,*,forward
691,9e:8d:de:80:29:28,*,*,*,scalews.withings.net,6,*,80,*,forward
692,*,9e:8d:de:80:29:28,*,scalews.withings.net,*,6,443,*,*,forward
693,*,9e:8d:de:80:29:28,*,scalews.withings.net,*,6,80,*,*,forward


In [4]:
column_list = ['sEth','dEth','typEth','proto','sPort',  'dPort', 'Source', 'Destination','action']
print(column_list)
state_counter = 1
state_mapping = {}
previous_column = ""
previous_column_list = []
previous_column_values = []
first_counter = True
df_list = []

['sEth', 'dEth', 'typEth', 'proto', 'sPort', 'dPort', 'Source', 'Destination', 'action']


In [5]:
for column in column_list:

    if first_counter:
        unique_values = raw_df[column].drop_duplicates().tolist()
        first_counter = False
        final_data = []
        for val in unique_values:
            #state_mapping[column+"___no_column___"+val] = "state-"+str(state_counter)
            state_mapping[val] = str(state_counter)
            ##print("state Mapping for 1st column:",state_mapping)
            final_data.append({column : val, "state": "state-"+str(state_counter)})
            ##print("final data for 1st column:",final_data)
            state_counter = state_counter + 1
            ##print("state counter for 1st column:",state_counter)
        final_df = pd.DataFrame.from_dict(final_data)
        ##print("final df for 1st column:",final_df)
        df_list.append(final_df)
        ##print("df_list for 1st column :",df_list)
        previous_column = column
        ##print("previous_column for 1st column :",previous_column)
        previous_column_list.append(column)
        ##print("previous_column_list for 1st column :",previous_column_list)

    else:
        # combined_df = (raw_df[previous_column].map(str) + '___' + raw_df[column].map(str)).drop_duplicates().tolist()
        # li = "___".join(i for i in previous_column_list)
        # print(li)
        # final_list = previous_column_list
        combined_df = raw_df[previous_column_list + [column]].astype(str).apply('___'.join, axis=1).drop_duplicates().tolist()
        ##print("combined df in else block",combined_df)
        final_data = []
        for val in combined_df:
            val_split = val.split("___")
            #state_mapping[column + "___" + val] = "state-" + str(state_counter)
            state_mapping[val] = str(state_counter)
            #print("state mapping in else block:",state_mapping)
            #print("val split in else block:",val_split)
            #final_data.append({'___'.join(previous_column_list): '___'.join(val_split[:-1]), column: str(val_split[-1]), "state": "state-" + str(state_counter)})
            #print("join split in else block",state_mapping.get('___'.join(val_split[:-1])))
            if(column=='action'):
                final_data.append({"Previous State": state_mapping.get('___'.join(val_split[:-1])), column: str(val_split[-1])})
            else:
                final_data.append({"Previous State": state_mapping.get('___'.join(val_split[:-1])), column: str(val_split[-1]), "Next State": "state-" + str(state_counter)})
            #print("final data in else block:",final_data)
            state_counter = state_counter + 1
        final_df = pd.DataFrame.from_dict(final_data)
        df_list.append(final_df)
        previous_column = column
        previous_column_list.append(column)



In [6]:

first_row = True

writer = pd.ExcelWriter('Output_of_BDD_States.xlsx', engine='xlsxwriter')
workbook = writer.book
for dfl in df_list:
    columns = dfl.columns
    if first_row:
        sheet_name = columns[0]
        first_row = False
    else:
        sheet_name = columns[1]
    # print(dfl.head(500))
    worksheet = workbook.add_worksheet(sheet_name)
    writer.sheets[sheet_name] = worksheet
    dfl.to_excel(writer, sheet_name=sheet_name, startrow=0, startcol=0)
writer.close()